In [19]:
import subprocess

def get_cuda_status():
    """
    Retrieves and displays CUDA status information using `nvidia-smi`.

    Returns:
        str: A string containing the output of `nvidia-smi`, or an error message.
    """
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error executing nvidia-smi: {e.stderr}"
    except FileNotFoundError:
        return "nvidia-smi not found. Make sure NVIDIA drivers and CUDA are installed."
    except Exception as e:
        return f"An unexpected error occurred: {e}"


def get_cuda_version():
    """
    Retrieves the CUDA version using `nvcc --version`.

    Returns:
        str: The CUDA version string, or an error message.
    """
    try:
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error executing nvcc --version: {e.stderr}"
    except FileNotFoundError:
        return "nvcc not found. Make sure NVIDIA CUDA Toolkit is installed."
    except Exception as e:
        return f"An unexpected error occurred: {e}"

def check_cuda_availability():
    """
    Checks if CUDA is available by attempting to import torch.cuda (if torch is installed)
    or by checking for nvidia-smi.

    Returns:
        bool: True if CUDA is likely available, False otherwise.
        str: A string with more details.
    """

    try:
        import torch
        if torch.cuda.is_available():
            return True, f"CUDA is available. Device: {torch.cuda.get_device_name(0)}"
        else:
            return False, "CUDA is not available according to torch."
    except ImportError:
        # torch is not installed, fall back to nvidia-smi
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        if result.returncode == 0:
            return True, "CUDA is likely available (nvidia-smi found), but torch is not installed to confirm."
        else:
            return False, "CUDA is not available (nvidia-smi not found)."
    except Exception as e:
        return False, f"An unexpected error occurred while checking CUDA availability: {e}"

if __name__ == "__main__":
    available, availability_message = check_cuda_availability()
    print(f"CUDA Availability: {available}")
    print(availability_message)

    print("\nCUDA Version:")
    print(get_cuda_version())

    print("\nNVIDIA-SMI Output:")
    print(get_cuda_status())

CUDA Availability: True
CUDA is available. Device: NVIDIA GeForce RTX 4090

CUDA Version:
nvcc not found. Make sure NVIDIA CUDA Toolkit is installed.

NVIDIA-SMI Output:
Sun Mar  2 03:03:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        On  |   00000000:01:00.0  On |                  Off |
| 33%   59C    P2            168W /  450W |   17099MiB /  2456

In [20]:
import subprocess
import os

def clean_cuda_cache():
    """
    Attempts to clean the CUDA cache by deleting the shader cache directories.
    This is a platform-specific operation and might require administrator privileges.
    """
    try:
        if os.name == 'nt':  # Windows
            cache_dir = os.path.join(os.environ.get('LOCALAPPDATA'), 'NVIDIA', 'DXCache')
            if os.path.exists(cache_dir):
                subprocess.run(['powershell', '-Command', f'Remove-Item -Recurse -Force "{cache_dir}"'], check=True)
                print(f"CUDA cache cleaned (Windows): {cache_dir}")
            else:
                print(f"CUDA cache directory not found (Windows): {cache_dir}")

            cache_dir = os.path.join(os.environ.get('LOCALAPPDATA'), 'NVIDIA', 'GLCache')
            if os.path.exists(cache_dir):
                subprocess.run(['powershell', '-Command', f'Remove-Item -Recurse -Force "{cache_dir}"'], check=True)
                print(f"CUDA cache cleaned (Windows): {cache_dir}")
            else:
                print(f"CUDA cache directory not found (Windows): {cache_dir}")

        elif os.name == 'posix':  # Linux/macOS
            cache_dir = os.path.expanduser('~/.nv/ComputeCache')
            if os.path.exists(cache_dir):
                subprocess.run(['rm', '-rf', cache_dir], check=True)
                print(f"CUDA cache cleaned (Linux/macOS): {cache_dir}")
            else:
                print(f"CUDA cache directory not found (Linux/macOS): {cache_dir}")

            cache_dir = os.path.expanduser('~/.cache/nvidia/GLCache')
            if os.path.exists(cache_dir):
                subprocess.run(['rm', '-rf', cache_dir], check=True)
                print(f"CUDA cache cleaned (Linux/macOS): {cache_dir}")
            else:
                print(f"CUDA cache directory not found (Linux/macOS): {cache_dir}")

        else:
            print("Unsupported operating system.")

    except subprocess.CalledProcessError as e:
        print(f"Error cleaning CUDA cache: {e}")
    except FileNotFoundError:
        print("rm or powershell not found")
    except PermissionError:
        print("Permission denied when trying to delete the cache. You may need administrator privileges.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


if __name__ == "__main__":
    clean_cuda_cache()

CUDA cache cleaned (Linux/macOS): /home/jags/.nv/ComputeCache
CUDA cache directory not found (Linux/macOS): /home/jags/.cache/nvidia/GLCache


In [ ]:
# ## Prerequisite. Run on Terminal. Install miniconda. Navigate to the folder cd /home/asus/

# mkdir -p ~/miniconda3
# wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
# bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
# rm -rf ~/miniconda3/miniconda.sh
# ~/miniconda3/bin/conda init bash
# ~/miniconda3/bin/conda init zsh

In [ ]:
#### RUn on terminal: Create a virtual environment, Install all the requirements like pytorch, xformers, cuda etc.
#### install the unsloth library and other libraries viz. peft

# conda create --name unsloth_env python=3.11 pytorch-cuda=12.1 pytorch cudatoolkit xformers -c pytorch -c nvidia -c xformers -y
# conda activate unsloth_env
# pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# pip install --no-deps trl peft accelerate bitsandbytes

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Mistral patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [6]:
alpaca_prompt = """You are an assistant specialized in generating detailed bug reports.

### Instruction:
Please create a bug report that
includes the following sections:
1. Steps to Reproduce (S2R): Detailed steps to replicate the issue.
2. Expected Result (ER): What you expected to happen.
3. Actual Result (AR): What actually happened.
4. Additional Information: Include relevant details such as software version, build number, environment, etc.
 Highlight the missing information to the reporter: Explicitly notify the user which information is missing.

### Context:
{Summary}

### Response:
{Response}
"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    texts = []
    for summary, report in zip(examples["LLAMA Output"], examples["Input"]):
        formatted_text = alpaca_prompt.format(Summary=summary,Response=report) + EOS_TOKEN
        texts.append(formatted_text)
    return {"text": texts}


In [7]:
import json
def formatting_prompts_func(examples):
    texts = []
    
    for summary, report in zip(examples["NEW_llama_output"], examples["text"]):
        # Create the conversation in the required JSON format
        conversation = [
            {"role": "system", "content": "You are an assistant specialized in generating detailed bug reports."},
            {"role": "user", "content": f"""### Instruction:
Please create a bug report that
includes the following sections:
1. Steps to Reproduce (S2R): Detailed steps to replicate the issue.
2. Expected Result (ER): What you expected to happen.
3. Actual Result (AR): What actually happened.
4. Additional Information: Include relevant details such as software version, build number, environment, etc.
Highlight the missing information to the reporter: Explicitly notify the user which information is missing.

### Context:
{summary}"""},
            {"role": "assistant", "content": report}
        ]
        
        # Convert each message to a JSON string and join with newlines
        formatted_text = "\n".join(json.dumps(message) for message in conversation)
        texts.append(formatted_text)
    
    return {"train_text": texts}

# Define the EOS token to be used with the tokenizer
EOS_TOKEN = tokenizer.eos_token

In [5]:
# alpaca_prompt= """Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.

# ### sql_prompt:
# {}

# ### sql:
# {}

# ### Explanation:
# {}"""

In [8]:
alpaca_prompt

'You are an assistant specialized in generating detailed bug reports.\n\n### Instruction:\nPlease create a bug report that\nincludes the following sections:\n1. Steps to Reproduce (S2R): Detailed steps to replicate the issue.\n2. Expected Result (ER): What you expected to happen.\n3. Actual Result (AR): What actually happened.\n4. Additional Information: Include relevant details such as software version, build number, environment, etc.\n Highlight the missing information to the reporter: Explicitly notify the user which information is missing.\n\n### Context:\n{Summary}\n\n### Response:\n{Response}\n'

In [25]:
# def formatting_prompts_func(examples):
#     sql_prompt = examples["sql_prompt"]
#     sql = examples["sql"]
#     sql_explanation = examples['sql_explanation']

#     texts = []
    
#     EOS_TOKEN = tokenizer.eos_token
#     for sql_prompt, sql, sql_explanation in zip(sql_prompt, sql, sql_explanation):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(sql_prompt, sql, sql_explanation) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }
# pass

In [14]:
from datasets import load_dataset
train_dataset = load_dataset("jag2023/Ultimate_CTQRS_good_BG", split = "train")
test_dataset = load_dataset("jag2023/Ultimate_CTQRS_good_BG", split = "test")
print(train_dataset.column_names)

['text', 'user_behviou', 'total_score', 'max_possible', 'score_percentage', 'RM1_size_passed', 'RM1_size_score', 'RM2_readability_passed', 'RM2_readability_score', 'RM3_punctuation_passed', 'RM3_punctuation_score', 'RM4_sentence_length_passed', 'RM4_sentence_length_score', 'RR1_itemization_passed', 'RR1_itemization_score', 'RR2_itemization_symbol_passed', 'RR2_itemization_symbol_score', 'RR3_environment_passed', 'RR3_environment_score', 'RR4_screenshot_passed', 'RR4_screenshot_score', 'RR5_screenshot_guideline_passed', 'RR5_screenshot_guideline_score', 'RA1_interface_element_passed', 'RA1_interface_element_score', 'RA2_user_behavior_passed', 'RA2_user_behavior_score', 'RA3_system_defect_passed', 'RA3_system_defect_score', 'RA4_defect_description_passed', 'RA4_defect_description_score', 'NEW_llama_output']


In [33]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("jag2023/CTQRS_dataset_final", split="train")

# Split the dataset into 90% train and 10% test
split_dataset = dataset.train_test_split(test_size=0.05)

# Extract the train and test datasets

train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

# Display the sizes of train and test datasets
print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


Train dataset size: 2781
Test dataset size: 147


In [15]:
train_dataset

Dataset({
    features: ['text', 'user_behviou', 'total_score', 'max_possible', 'score_percentage', 'RM1_size_passed', 'RM1_size_score', 'RM2_readability_passed', 'RM2_readability_score', 'RM3_punctuation_passed', 'RM3_punctuation_score', 'RM4_sentence_length_passed', 'RM4_sentence_length_score', 'RR1_itemization_passed', 'RR1_itemization_score', 'RR2_itemization_symbol_passed', 'RR2_itemization_symbol_score', 'RR3_environment_passed', 'RR3_environment_score', 'RR4_screenshot_passed', 'RR4_screenshot_score', 'RR5_screenshot_guideline_passed', 'RR5_screenshot_guideline_score', 'RA1_interface_element_passed', 'RA1_interface_element_score', 'RA2_user_behavior_passed', 'RA2_user_behavior_score', 'RA3_system_defect_passed', 'RA3_system_defect_score', 'RA4_defect_description_passed', 'RA4_defect_description_score', 'NEW_llama_output'],
    num_rows: 3172
})

In [13]:
dataset = train_dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/3172 [00:00<?, ? examples/s]

In [36]:
dataset

Dataset({
    features: ['NEW_llama_output', 'raw_text', 'total_score', 'text'],
    num_rows: 2781
})

In [37]:
dataset["text"][0:1]

['{"role": "system", "content": "You are an assistant specialized in generating detailed bug reports."}\n{"role": "user", "content": "### Instruction:\\nPlease create a bug report that\\nincludes the following sections:\\n1. Steps to Reproduce (S2R): Detailed steps to replicate the issue.\\n2. Expected Result (ER): What you expected to happen.\\n3. Actual Result (AR): What actually happened.\\n4. Additional Information: Include relevant details such as software version, build number, environment, etc.\\nHighlight the missing information to the reporter: Explicitly notify the user which information is missing.\\n\\n### Context:\\nHere\'s the rewritten bug report:\\n\\n\\"So, I\'m using Firefox in French and trying to right-click on an image on a webpage - nothing out of the ordinary. But when I press \'s\', which is supposed to open up that context menu with all sorts of options, it just doesn\'t do anything. And if you look closely, there are two items in that menu that both have the l

In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "train_text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2,
        warmup_steps = 3,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps =  120,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/3172 [00:00<?, ? examples/s]

In [17]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.988 GB.
4.031 GB of memory reserved.


In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,172 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 1,586
 "-____-"     Number of trainable parameters = 41,943,040


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [41]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

1087.205 seconds used for training.
18.12 minutes used for training.
Peak reserved memory = 8.195 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 34.163 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [44]:
from datasets import load_dataset

# Load the dataset
test_dataset = load_dataset("jag2023/CTQRS_test", split="test")

# # Split the dataset into 90% train and 10% test
# split_dataset = dataset.train_test_split(test_size=0.05)

# # Extract the train and test datasets

# train_dataset = split_dataset['train']
# test_dataset = split_dataset['test']

# # Display the sizes of train and test datasets
# print(f"Train dataset size: {len(train_dataset)}")
# print(f"Test dataset size: {len(test_dataset)}")


In [45]:
test_dataset

Dataset({
    features: ['_id_x', 'count', 'id', 'tags', 'reactions', 'is_private', 'raw_text', 'creation_time_x', 'text', 'bug_id', 'attachment_id', 'creator_x', 'time', 'author', 'Bug report', 'author_id', '_id_y', 'severity', 'summary', 'votes', 'cf_tracking_firefox_esr128', 'qa_contact', 'cf_fx_points', 'cf_last_resolved', 'duplicates', 'see_also', 'cf_tracking_thunderbird_esr128', 'cf_tracking_thunderbird_esr115', 'cc_detail', 'cf_qa_whiteboard', 'blocks', 'cf_webcompat_score', 'product', 'flags', 'alias', 'creation_time_y', 'cf_has_str', 'mentors', 'creator_detail', 'cf_status_thunderbird_esr128', 'cf_crash_signature', 'op_sys', 'cf_accessibility_severity', 'target_milestone', 'url', 'cf_user_story', 'cf_tracking_firefox_relnote', 'cf_webcompat_priority', 'priority', 'cf_fx_iteration', 'cf_a11y_review_project_flag', 'is_open', 'cf_tracking_firefox132', 'status', 'cf_cab_review', 'cf_status_firefox134', 'assigned_to_detail', 'cf_tracking_firefox134', 'version', 'keywords', 'cf_sta

In [51]:
import pandas as pd

# Assuming model and tokenizer are already loaded and set up, e.g.:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("model_name")
# model = AutoModelForCausalLM.from_pretrained("model_name")
# model.to("cuda")
# FastLanguageModel.for_inference(model)  # If using a specific library for faster inference

# Define the prompt template
alpaca_prompt = """You are an senior engineer specialized in generating detailed bug reports.

### Instruction:
Please create a bug report with proper itemization and it should includes the following sections:
1. Steps to Reproduce (S2R): Detailed steps to replicate the issue.
2. Expected Result (ER): What you expected to happen.
3. Actual Result (AR): What actually happened.
4. Additional Information: Include relevant details such as software version, build number, environment, etc.
Highlight the missing information to the reporter if software version, build number, environment, etc not present.
### Context:
{Summary}

### Response:
{Response}
"""

# Load the dataset
dataset = test_dataset.to_pandas()
dataset= dataset[:200]

results = []

for index, row in dataset.iterrows():
    summary = row['NEW_llama_output']
    actual_report = row['raw_text']
    # mistral_repo = row['Mistral Report']
    # pure_llama = row['Output']
    FastLanguageModel.for_inference(model)
    
    # Format the prompt with the summary
    prompt = alpaca_prompt.format(Summary=summary, Response="")
    
    # Tokenize the prompt
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    # Generate the output
    outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
    
    # Decode the output
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    import markdown
    # html_response = markdown.markdown(decoded_output)
    # print(html_response)
    
    
    # # Extract the generated response (the part after "### Response:\n")
    response_start = decoded_output.find("### Response:\n") + len("### Response:\n")
    generated_response = decoded_output[response_start:].strip()
    
    # Collect the results
    print("index",index)
    # print("summary",summary[:10])
    results.append({
        'Summary': summary,
        'Actual Report': actual_report,
        # 'Mistral Report': mistral_repo,
        'agent_Fine_tune mistral Output_': generated_response,
        # 'Pura llama Output': pure_llama,
    })

# Create DataFrame from results
results_df = pd.DataFrame(results)

# Save to Excel
results_df.to_excel("CTQRS_200_Score_test_mistral.xlsx", index=False)

index 0
index 1
index 2
index 3
index 4
index 5
index 6
index 7
index 8
index 9
index 10
index 11
index 12
index 13
index 14
index 15
index 16
index 17
index 18
index 19
index 20
index 21
index 22
index 23
index 24
index 25
index 26
index 27
index 28
index 29
index 30
index 31
index 32
index 33
index 34
index 35
index 36
index 37
index 38
index 39
index 40
index 41
index 42
index 43
index 44
index 45
index 46
index 47
index 48
index 49
index 50
index 51
index 52
index 53
index 54
index 55
index 56
index 57
index 58
index 59
index 60
index 61
index 62
index 63
index 64
index 65
index 66
index 67
index 68
index 69
index 70
index 71
index 72
index 73
index 74
index 75
index 76
index 77
index 78
index 79
index 80
index 81
index 82
index 83
index 84
index 85
index 86
index 87
index 88
index 89
index 90
index 91
index 92
index 93
index 94
index 95
index 96
index 97
index 98
index 99
index 100
index 101
index 102
index 103
index 104
index 105
index 106
index 107
index 108
index 109
index 110


In [ ]:
# # !pip install markdownfrom datasets import load_dataset, DatasetDict

# # Load the initial dataset
# dataset = load_dataset("jag2023/llama_bug_rep_Summary-dataset", split="train")

# # Perform the first split to get 80% train and 20% (test + validation)
# train_test_dataset = dataset.train_test_split(test_size=0.2, seed=42) # Setting a seed for reproducibility

# # Split the 20% (test + validation) into 10% test and 10% validation
# test_valid_dataset = train_test_dataset['test'].train_test_split(test_size=0.5, seed=42) # Using the same seed for consistency

# # Create the final train, validation, and test datasets
# train_dataset = train_test_dataset['train']
# validation_dataset = test_valid_dataset['train']
# test_dataset = test_valid_dataset['test']

# # Print dataset information to verify the splits (optional but good practice)
# print(f"Train dataset size: {len(train_dataset)}")
# print(f"Validation dataset size: {len(validation_dataset)}")
# print(f"Test dataset size: {len(test_dataset)}")

# # Print column names for the train dataset (as in the original code)
# print("\nColumn names of the train dataset:")
# print(train_dataset.column_names)

Train dataset size: 4891
Validation dataset size: 611
Test dataset size: 612

Column names of the train dataset:
['Input', 'LLAMA Output', 'SBERT', 'BLEU', 'ROUGE', 'METEOR', 'Jaccard', 'Cosine']


## RQ2

In [57]:
import pandas as pd
df = pd.read_excel("27_feb_8pm_ollama_summary_RQ2.xlsx")

In [ ]:
import pandas as pd

# Assuming model and tokenizer are already loaded and set up, e.g.:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("model_name")
# model = AutoModelForCausalLM.from_pretrained("model_name")
# model.to("cuda")
# FastLanguageModel.for_inference(model)  # If using a specific library for faster inference

# Define the prompt template
alpaca_prompt = """You are an assistant specialized in generating detailed bug reports.

### Instruction:
Please create a bug report that includes the following sections:
1. Steps to Reproduce (S2R): Detailed steps to replicate the issue.
2. Expected Result (ER): What you expected to happen.
3. Actual Result (AR): What actually happened.
4. Additional Information: Include relevant details such as software version, build number, environment, etc.
Highlight the missing information to the reporter if software version, build number, environment, etc not present.
### Context:
{Summary}

### Response:
{Response}
"""

# Load the dataset
# dataset = test_dataset.to_pandas()
dataset= df2

results = []

for index, row in dataset.iterrows():
    summary = row['Summary']
    actual_report = row['Actual Report']
    # mistral_repo = row['Mistral Report']
    # pure_llama = row['Output']
    FastLanguageModel.for_inference(model)
    
    # Format the prompt with the summary
    prompt = alpaca_prompt.format(Summary=summary, Response="")
    
    # Tokenize the prompt
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    # Generate the output
    outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
    
    # Decode the output
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    import markdown
    # html_response = markdown.markdown(decoded_output)
    # print(html_response)
    
    
    # # Extract the generated response (the part after "### Response:\n")
    response_start = decoded_output.find("### Response:\n") + len("### Response:\n")
    generated_response = decoded_output[response_start:].strip()
    
    # Collect the results
    print("index",index)
    # print("summary",summary[:10])
    results.append({
        'Summary': summary,
        'Actual Report': actual_report,
        # 'Mistral Report': mistral_repo,
        'agent_Fine_tune llama Output_': generated_response,
        # 'Pura llama Output': pure_llama,
    })

# Create DataFrame from results
results_df = pd.DataFrame(results)

# Save to Excel
results_df.to_excel("top_good50_mapping_study.xlsx", index=False)

In [40]:
df

,text,NEW_llama_output
0,created attachment 9303641\nscreenshot\n\n[tra...,I was trying to use the Japanese UI build of F...
1,user agent: mozilla/5.0 (macintosh; intel mac ...,I was trying to download Ubuntu for my ARM-bas...
2,NaN,"I was trying to use a new feature on my app, s..."
3,created attachment 9303207\nfirefox-typed-slug...,Here's the bug report transformed into a conci...
4,steps to reproduce:\n\n1. export .vcf file fro...,I'm still fuming about this frustrating experi...
5,created attachment 9302717\nscrfra1.pdf\n\nuse...,I'm still frustrated about this issue I had wi...
6,created attachment 9210007\nclose tabs.gif\n\n...,"So, I was using Firefox on my Mac, Windows, an..."
7,user agent: mozilla/5.0 (windows nt 10.0; win6...,Here's the rewritten paragraph:\n\nI'm still s...
8,user agent: mozilla/5.0 (windows nt 10.0; win6...,I was working in a Windows HCM theme and I wan...
9,**affected versions**\n* latest nightly 88.0a1...,Here's the rewritten paragraph:\n\nI'm still t...


In [62]:
import pandas as pd

# Assuming model and tokenizer are already loaded and set up, e.g.:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("model_name")
# model = AutoModelForCausalLM.from_pretrained("model_name")
# model.to("cuda")
# FastLanguageModel.for_inference(model)  # If using a specific library for faster inference

# Define the prompt template
alpaca_prompt = """You are an assistant specialized in generating detailed bug reports.

### Instruction:
Please create a bug report that includes the following sections:
1. Steps to Reproduce (S2R): Detailed steps to replicate the issue.
2. Expected Result (ER): What you expected to happen.
3. Actual Result (AR): What actually happened.
4. Additional Information: Include relevant details such as software version, build number, environment, etc.
Highlight the missing information to the reporter if software version, build number, environment, etc not present.
### Context:
{Summary}

### Response:
{Response}
"""

# Load the dataset
# dataset = test_dataset.to_pandas()
dataset= df2

results = []

for index, row in dataset.iterrows():
    summary = row['Summary']
    actual_report = row['Actual Report']
    # mistral_repo = row['Mistral Report']
    # pure_llama = row['Output']
    FastLanguageModel.for_inference(model)
    
    # Format the prompt with the summary
    prompt = alpaca_prompt.format(Summary=summary, Response="")
    
    # Tokenize the prompt
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    # Generate the output
    outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
    
    # Decode the output
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    import markdown
    # html_response = markdown.markdown(decoded_output)
    # print(html_response)
    
    
    # # Extract the generated response (the part after "### Response:\n")
    response_start = decoded_output.find("### Response:\n") + len("### Response:\n")
    generated_response = decoded_output[response_start:].strip()
    
    # Collect the results
    print("index",index)
    # print("summary",summary[:10])
    results.append({
        'Summary': summary,
        'Actual Report': actual_report,
        # 'Mistral Report': mistral_repo,
        'agent_Fine_tune llama Output_': generated_response,
        # 'Pura llama Output': pure_llama,
    })

# Create DataFrame from results
results_df = pd.DataFrame(results)

# Save to Excel
results_df.to_excel("top_good50_mapping_study.xlsx", index=False)

index 0
index 1
index 2
index 3
index 4
index 5
index 6
index 7
index 8
index 9
index 10
index 11
index 12
index 13
index 14
index 15
index 16
index 17
index 18
index 19
index 20
index 21
index 22
index 23
index 24
index 25
index 26
index 27
index 28
index 29
index 30
index 31
index 32
index 33
index 34
index 35
index 36
index 37
index 38
index 39
index 40
index 41
index 42
index 43
index 44
index 45
index 46
index 47
index 48
index 49


In [53]:
df2 = pd.read_excel("top_good50_mapping_study.xlsx")

In [54]:
df2

,Summary,Actual Report,Fine_tune llama Output
0,I was trying to use the Japanese UI build of F...,created attachment 9303641\nscreenshot\n\n[tra...,**Bug 1779695:** Introduce modern flex on in-c...
1,I was trying to download Ubuntu for my ARM-bas...,user agent: mozilla/5.0 (macintosh; intel mac ...,**Bug Report**\n\n**1. Steps to Reproduce (S2R...
2,"I was trying to use a new feature on my app, s...",NaN,**Bug Report**\n\n**Steps to Reproduce (S2R):*...
3,user agent: mozilla/5.0 (windows nt 10.0; win6...,created attachment 9303207\nfirefox-typed-slug...,**Bug ID:** [To be assigned]\n\n**Affected Ver...
4,I'm still fuming about this frustrating experi...,steps to reproduce:\n\n1. export .vcf file fro...,**Bug Report**\n\n**Steps to Reproduce (S2R):*...
5,\nuser agent: mozilla/5.0 (windows nt 10.0; wi...,created attachment 9302717\nscrfra1.pdf\n\nuse...,**Bug ID:** [To be assigned]\n\n**Software Ver...
6,"So, I was using Firefox on my Mac, Windows, an...",created attachment 9210007\nclose tabs.gif\n\n...,"**Bug Report**\n\n**Platform:** macOS 10.15.7,..."
7,user agent: mozilla/5.0 (windows nt 10.0; win6...,user agent: mozilla/5.0 (windows nt 10.0; win6...,**Bug Report**\n\n**Title:** Firefox browser d...
8,user agent: mozilla/5.0 (windows nt 10.0; win6...,user agent: mozilla/5.0 (windows nt 10.0; win6...,**Bug Report**\n\n**Title:** Date and Time con...
9,Here's the rewritten paragraph:\n\nI'm still t...,**affected versions**\n* latest nightly 88.0a1...,**Bug Report**\n\n**Steps to Reproduce (S2R):*...


In [55]:
# import re
# import pandas as pd # Make sure you have pandas imported

# def structure_llm_output(llm_output_text):
#     """
#     Structures the LLM output text to extract sections for bug report.
#     ... (rest of your structure_llm_output function code is the same) ...
#     """
#     sections = {
#         "Steps to Reproduce": None,
#         "Expected Result": None,
#         "Actual Result": None,
#         "Additional Information": None,
#     }
#     section_headings = list(sections.keys())
#     for i, section_name in enumerate(section_headings):
#         next_section_heading = section_headings[i+1] if i+1 < len(section_headings) else None
#         # start_pattern = re.compile(rf"{re.escape(section_name)}:\s*(.*?)(?=(?:\n{re.escape(section_headings[i+1])}:)|\Z)", re.DOTALL)
#         start_pattern = re.compile(rf"{re.escape(section_name)}:\s*(.*?)(?=(?:\n{re.escape(section_headings[i+1])}:)|\Z)", re.DOTALL)
#         match = start_pattern.search(llm_output_text)
#         if match:
#             sections[section_name] = match.group(1).strip()
#     return sections

# # Assuming df2 is your DataFrame and 'Fine_tune llama Output' is the column
# # --- Process DataFrame ---

# # 1. Create a new column to store the structured output (dictionaries)
# df2['Structured_Output'] = None # Initialize a new column

# # 2. Iterate through each row of df2 and apply structure_llm_output to the 'Fine_tune llama Output'
# for index, row in df2.iterrows():
#     llm_output_text = row['Fine_tune llama Output'] # Get the string from the current row
#     if isinstance(llm_output_text, str): # Important: Check if it's actually a string
#         structured_output_dict = structure_llm_output(llm_output_text)
#         df2.at[index, 'Structured_Output'] = structured_output_dict # Store the dictionary back in the DataFrame
#     else:
#         df2.at[index, 'Structured_Output'] = None # Or handle non-string cases as needed, perhaps log a warning

# # --- Example of accessing structured data ---
# # After processing, each row in df2 will have a dictionary in the 'Structured_Output' column

# # Example: Print structured output for the first few rows
# print("Structured Output for DataFrame:")
# for index, row in df2.head().iterrows(): # Show for the first few rows
#     structured_data = row['Structured_Output']
#     if structured_data: # Check if structured data was successfully extracted
#         print(f"\nRow Index: {index}")
#         print("Steps:", structured_data.get("Steps to Reproduce"))
#         print("Expected:", structured_data.get("Expected Result"))
#         print("Actual:", structured_data.get("Actual Result"))
#         print("Additional Info:", structured_data.get("Additional Information"))
#     else:
#         print(f"\nRow Index: {index} - No structured output (perhaps input was not a string or parsing failed)")
#     print("-" * 30)

In [56]:
import pandas as pd

# Assuming model and tokenizer are already loaded and set up, e.g.:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("model_name")
# model = AutoModelForCausalLM.from_pretrained("model_name")
# model.to("cuda")
# FastLanguageModel.for_inference(model)  # If using a specific library for faster inference

# Define the prompt template
alpaca_prompt = """You are an assistant specialized in generating detailed bug reports.

### Instruction:
Please create a bug report that includes the following sections:
1. Steps to Reproduce (S2R): Detailed steps to replicate the issue.
2. Expected Result (ER): What you expected to happen.
3. Actual Result (AR): What actually happened.
4. Additional Information: Include relevant details such as software version, build number, environment, etc.
Highlight the missing information to the reporter if software version, build number, environment, etc not present.
### Context:
{Summary}

### Response:
{Response}
"""

# Load the dataset
# dataset = test_dataset.to_pandas()
dataset= df

results = []

for index, row in dataset.iterrows():
    summary = row['Summary']
    actual_report = row['Actual Report']
    mistral_repo = row['Mistral Report']
    pure_llama = row['Output']
    FastLanguageModel.for_inference(model)
    
    # Format the prompt with the summary
    prompt = alpaca_prompt.format(Summary=summary, Response="")
    
    # Tokenize the prompt
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    # Generate the output
    outputs = model.generate(**inputs, max_new_tokens=1024, use_cache=True)
    
    # Decode the output
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    import markdown
    # html_response = markdown.markdown(decoded_output)
    # print(html_response)
    
    
    # # Extract the generated response (the part after "### Response:\n")
    response_start = decoded_output.find("### Response:\n") + len("### Response:\n")
    generated_response = decoded_output[response_start:].strip()
    
    # Collect the results
    print("index",index)
    # print("summary",summary[:10])
    results.append({
        'Summary': summary,
        'Actual Report': actual_report,
        'Mistral Report': mistral_repo,
        'Fine_tune Output': generated_response,
        'Pura llama Output': pure_llama,
    })

# Create DataFrame from results
results_df = pd.DataFrame(results)

# Save to Excel
results_df.to_excel("Full_Final_epoc_600_newrefine_llama_Fine_tine_output.xlsx", index=False)

KeyError: 'Summary'

In [16]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [13]:
import torch
torch.cuda.empty_cache()


In [18]:
model.save_pretrained_gguf("model", tokenizer, quantization_method = "f32")

make: Entering directory '/home/jags/research/Final_Paper_fine_tune/llama.cpp'
make: Leaving directory '/home/jags/research/Final_Paper_fine_tune/llama.cpp'
-- The C compiler identification is GNU 13.3.0


Makefile:2: *** The Makefile build is deprecated. Use the CMake build instead. For more details, see https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md.  Stop.


-- The CXX compiler identification is GNU 13.3.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Git: /usr/bin/git (found version "2.43.0") 
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- Including CPU backend
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- x86 detected
-- Adding CPU backend

CMake Error at /usr/share/cmake-3.28/Modules/FindPackageHandleStandardArgs.cmake:230 (message):
  Could NOT find CURL (missing: CURL_LIBRARY CURL_INCLUDE_DIR)
Call Stack (most recent call first):
  /usr/share/cmake-3.28/Modules/FindPackageHandleStandardArgs.cmake:600 (_FPHSA_FAILURE_MESSAGE)
  /usr/share/cmake-3.28/Modules/FindCURL.cmake:194 (find_package_handle_standard_args)
  common/CMakeLists.txt:88 (find_package)




RuntimeError: *** Unsloth: Failed compiling llama.cpp using os.system(...) with error 256. Please report this ASAP!

Follow this Steps for Local Saving

In [19]:
model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit",)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.06 out of 15.47 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 91%|█████████ | 29/32 [00:01<00:00, 28.82it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [00:01<00:00, 16.82it/s]


Unsloth: Saving tokenizer... Done.
Done.


In [20]:
!git clone --recursive https://github.com/ggerganov/llama.cpp

fatal: destination path 'llama.cpp' already exists and is not an empty directory.


In [21]:
!make clean -C llama.cpp

make: Entering directory '/home/jags/research/Final_Paper_fine_tune/llama.cpp'
Makefile:2: *** The Makefile build is deprecated. Use the CMake build instead. For more details, see https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md.  Stop.
make: Leaving directory '/home/jags/research/Final_Paper_fine_tune/llama.cpp'


In [22]:
!make all -j -C llama.cpp

make: Entering directory '/home/jags/research/Final_Paper_fine_tune/llama.cpp'
Makefile:2: *** The Makefile build is deprecated. Use the CMake build instead. For more details, see https://github.com/ggml-org/llama.cpp/blob/master/docs/build.md.  Stop.
make: Leaving directory '/home/jags/research/Final_Paper_fine_tune/llama.cpp'


In [23]:
!pip install gguf protobuf

In [27]:
!python llama.cpp/convert_hf_to_gguf.py merged_model --outfile jagunsloth_model_8.gguf --outtype q8_0

INFO:hf-to-gguf:Loading model: merged_model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> Q8_0, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.bfloat16 --> Q8_0, shape = {14336, 4096}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.bfloat16 --> Q8_0, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.bfloat16 --> Q8_0, shape = {4096, 14336}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {4096}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.bfloat16 --> Q8_0, shape = {4096, 1024}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.bfloat16 --> Q

INSTALL OLLAMA using Terminal

In [33]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
[sudo] password for asus: 


In [52]:
!ollama create jags -f Modelfile

gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠼ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠏ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠦ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠋ gathering model components ⠙ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠴ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠋ gathering model components ⠙ gathering model components ⠸ gathering model components ⠼ gathering mode

In [20]:
alpaca_prompt1= """Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.

### sql_prompt:
{}

### sql:
{}

### Explanation:
{}"""

In [21]:
input = alpaca_prompt1.format(
    "What is the total volume of timber sold by each salesperson, sorted by salesperson?",
    "",
    "",)

In [22]:
input

'Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\nWhat is the total volume of timber sold by each salesperson, sorted by salesperson?\n\n### sql:\n\n\n### Explanation:\n'

In [26]:
query = "What is the average property size in inclusive housing areas?"

In [27]:
input = alpaca_prompt1.format(
    query,
    "",
    "",)

In [28]:
input

'Below is an instruction that give an sql prompt. Write a response that appropriately completes the request and gives you an sql and the corresponding explanation.\n\n### sql_prompt:\nWhat is the average property size in inclusive housing areas?\n\n### sql:\n\n\n### Explanation:\n'